# scikit-learn 中級チュートリアル

このチュートリアルでは、scikit-learn の発展的な機能を学びます。アンサンブル学習、ハイパーパラメータチューニング、パイプライン、クラスタリングなどを習得しましょう。

## 学習内容
1. アンサンブル学習（ランダムフォレスト、勾配ブースティング）
2. ハイパーパラメータチューニング
3. パイプライン
4. 特徴量選択
5. クラスタリング
6. 次元削減
7. モデルの保存と読み込み

## 環境設定

In [ ]:
# JupyterLite 環境でのパッケージインストール
import sys
if 'pyodide' in sys.modules:
    import piplite
    await piplite.install(['numpy', 'pandas', 'matplotlib', 'scikit-learn'])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
print(f'scikit-learn version: {sklearn.__version__}')

# 警告を抑制
import warnings
warnings.filterwarnings('ignore')

# 乱数シード
np.random.seed(42)

In [ ]:
# サンプルデータの読み込み
from sklearn.datasets import load_iris, load_wine, load_diabetes, make_blobs, make_moons

iris = load_iris()
wine = load_wine()
diabetes = load_diabetes()

print('データセット読み込み完了')

---
## 1. アンサンブル学習

### 1.1 ランダムフォレスト

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# データ準備
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ランダムフォレスト
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
print(f'正解率: {accuracy_score(y_test, y_pred):.4f}')

In [ ]:
# 特徴量の重要度
importance = rf_model.feature_importances_

plt.figure(figsize=(8, 5))
plt.barh(iris.feature_names, importance)
plt.xlabel('Importance')
plt.title('Random Forest Feature Importance')
plt.show()

In [ ]:
# 木の数による精度の変化
n_estimators_range = [1, 5, 10, 20, 50, 100, 200]
scores = []

for n in n_estimators_range:
    model = RandomForestClassifier(n_estimators=n, random_state=42)
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

plt.figure(figsize=(8, 5))
plt.plot(n_estimators_range, scores, marker='o')
plt.xlabel('Number of Trees')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Number of Trees')
plt.show()

### 1.2 勾配ブースティング

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# 勾配ブースティング
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)
print(f'正解率: {accuracy_score(y_test, y_pred):.4f}')

In [ ]:
# 学習曲線（ステージごとの精度）
train_scores = []
test_scores = []

for i, y_pred_train in enumerate(gb_model.staged_predict(X_train)):
    train_scores.append(accuracy_score(y_train, y_pred_train))

for i, y_pred_test in enumerate(gb_model.staged_predict(X_test)):
    test_scores.append(accuracy_score(y_test, y_pred_test))

plt.figure(figsize=(10, 5))
plt.plot(train_scores, label='Train')
plt.plot(test_scores, label='Test')
plt.xlabel('Number of Boosting Stages')
plt.ylabel('Accuracy')
plt.title('Gradient Boosting Learning Curve')
plt.legend()
plt.show()

### 1.3 アンサンブル手法の比較

In [ ]:
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier,
    AdaBoostClassifier,
    BaggingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Bagging': BaggingClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

results = {}
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    results[name] = {'mean': scores.mean(), 'std': scores.std()}
    print(f'{name}: {scores.mean():.4f} (+/- {scores.std():.4f})')

In [ ]:
# 比較の可視化
names = list(results.keys())
means = [results[name]['mean'] for name in names]
stds = [results[name]['std'] for name in names]

plt.figure(figsize=(10, 5))
plt.bar(names, means, yerr=stds, capsize=5)
plt.ylabel('Accuracy')
plt.title('Ensemble Methods Comparison')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---
## 2. ハイパーパラメータチューニング

### 2.1 グリッドサーチ

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# パラメータグリッド
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}

# グリッドサーチ
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print('=== グリッドサーチ結果 ===')
print(f'最良パラメータ: {grid_search.best_params_}')
print(f'最良スコア: {grid_search.best_score_:.4f}')

In [ ]:
# 結果の詳細
results_df = pd.DataFrame(grid_search.cv_results_)
print('\n上位5件:')
print(results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
      .sort_values('rank_test_score')
      .head())

In [ ]:
# 最良モデルでテスト
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(f'テスト精度: {accuracy_score(y_test, y_pred):.4f}')

### 2.2 ランダムサーチ

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# パラメータ分布
param_distributions = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 15),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

# ランダムサーチ
random_search = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train, y_train)

print('=== ランダムサーチ結果 ===')
print(f'最良パラメータ: {random_search.best_params_}')
print(f'最良スコア: {random_search.best_score_:.4f}')

---
## 3. パイプライン

### 3.1 基本的なパイプライン

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# パイプラインの作成
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 学習
pipeline.fit(X_train, y_train)

# 予測
y_pred = pipeline.predict(X_test)
print(f'正解率: {accuracy_score(y_test, y_pred):.4f}')

### 3.2 パイプラインとグリッドサーチ

In [ ]:
# パイプラインのパラメータチューニング
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': [0.01, 0.1, 1]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(f'最良パラメータ: {grid_search.best_params_}')
print(f'最良スコア: {grid_search.best_score_:.4f}')

### 3.3 make_pipeline

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

# 簡潔な書き方
pipe = make_pipeline(
    StandardScaler(),
    PCA(n_components=2),
    LogisticRegression()
)

pipe.fit(X_train, y_train)
print(f'正解率: {pipe.score(X_test, y_test):.4f}')
print(f'パイプラインのステップ: {pipe.steps}')

### 3.4 ColumnTransformer

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# 混合データの作成
df = pd.DataFrame({
    'age': [25, 30, 35, 40, 45],
    'income': [30000, 50000, 70000, 90000, 110000],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'education': ['High School', 'Bachelor', 'Master', 'PhD', 'Bachelor']
})

print('混合データ:')
print(df)

In [ ]:
# 列ごとに異なる前処理
numeric_features = ['age', 'income']
categorical_features = ['gender', 'education']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# 変換
X_transformed = preprocessor.fit_transform(df)
print(f'変換後の形状: {X_transformed.shape}')
print(f'変換後のデータ:\n{X_transformed}')

---
## 4. 特徴量選択

### 4.1 分散による選択

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# サンプルデータ（低分散の特徴量を含む）
X_sample = np.array([[1, 0, 0.1],
                     [2, 0, 0.2],
                     [3, 0, 0.3],
                     [4, 0, 0.4],
                     [5, 0, 0.5]])

print(f'元のデータ形状: {X_sample.shape}')
print(f'各列の分散: {X_sample.var(axis=0)}')

# 分散が閾値以下の特徴量を削除
selector = VarianceThreshold(threshold=0.1)
X_selected = selector.fit_transform(X_sample)

print(f'選択後の形状: {X_selected.shape}')

### 4.2 単変量特徴量選択

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, chi2

# iris データセット
X = iris.data
y = iris.target

# 上位2つの特徴量を選択
selector = SelectKBest(f_classif, k=2)
X_selected = selector.fit_transform(X, y)

print(f'元の特徴量: {iris.feature_names}')
print(f'選択された特徴量: {np.array(iris.feature_names)[selector.get_support()]}')
print(f'各特徴量のスコア: {selector.scores_}')

In [ ]:
# スコアの可視化
plt.figure(figsize=(8, 5))
plt.bar(iris.feature_names, selector.scores_)
plt.xlabel('Features')
plt.ylabel('F-Score')
plt.title('Feature Importance (F-classif)')
plt.show()

### 4.3 再帰的特徴量削減（RFE）

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# RFE
model = LogisticRegression(max_iter=200)
rfe = RFE(model, n_features_to_select=2)
rfe.fit(X, y)

print('RFE による特徴量選択:')
for name, selected, ranking in zip(iris.feature_names, rfe.support_, rfe.ranking_):
    print(f'  {name}: 選択={selected}, ランク={ranking}')

---
## 5. クラスタリング

### 5.1 K-means

In [ ]:
from sklearn.cluster import KMeans

# サンプルデータ生成
X_cluster, y_true = make_blobs(n_samples=300, centers=4, random_state=42)

# K-means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
y_kmeans = kmeans.fit_predict(X_cluster)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 真のラベル
axes[0].scatter(X_cluster[:, 0], X_cluster[:, 1], c=y_true, cmap='viridis')
axes[0].set_title('True Labels')

# K-means結果
axes[1].scatter(X_cluster[:, 0], X_cluster[:, 1], c=y_kmeans, cmap='viridis')
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
                c='red', marker='X', s=200, label='Centroids')
axes[1].set_title('K-Means Clustering')
axes[1].legend()

plt.tight_layout()
plt.show()

### 5.2 エルボー法

In [ ]:
# 最適なクラスタ数を探す
inertias = []
K_range = range(1, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_cluster)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

### 5.3 シルエットスコア

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
K_range = range(2, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_cluster)
    score = silhouette_score(X_cluster, labels)
    silhouette_scores.append(score)
    print(f'K={k}: シルエットスコア = {score:.4f}')

plt.figure(figsize=(8, 5))
plt.plot(K_range, silhouette_scores, 'bo-')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score by K')
plt.show()

### 5.4 DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

# 月形のデータ
X_moons, y_moons = make_moons(n_samples=200, noise=0.05, random_state=42)

# K-means vs DBSCAN
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 真のラベル
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis')
axes[0].set_title('True Labels')

# K-means
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_kmeans = kmeans.fit_predict(X_moons)
axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_kmeans, cmap='viridis')
axes[1].set_title('K-Means (K=2)')

# DBSCAN
dbscan = DBSCAN(eps=0.2, min_samples=5)
labels_dbscan = dbscan.fit_predict(X_moons)
axes[2].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_dbscan, cmap='viridis')
axes[2].set_title('DBSCAN')

plt.tight_layout()
plt.show()

---
## 6. 次元削減

### 6.1 主成分分析（PCA）

In [ ]:
from sklearn.decomposition import PCA

# iris データセット
X = iris.data
y = iris.target

# 標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# 寄与率
print('各主成分の寄与率:')
for i, ratio in enumerate(pca.explained_variance_ratio_):
    print(f'  PC{i+1}: {ratio:.4f}')
print(f'累積寄与率: {pca.explained_variance_ratio_.cumsum()}')

In [ ]:
# 寄与率の可視化
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 各成分の寄与率
axes[0].bar(range(1, 5), pca.explained_variance_ratio_)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('Explained Variance by Component')

# 累積寄与率
axes[1].plot(range(1, 5), pca.explained_variance_ratio_.cumsum(), 'bo-')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95%')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('Cumulative Explained Variance')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 2次元にプロット
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='viridis')
plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.2%})')
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.2%})')
plt.title('PCA of Iris Dataset')
plt.colorbar(scatter, label='Species')
plt.show()

### 6.2 t-SNE

In [ ]:
from sklearn.manifold import TSNE

# t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

# PCA vs t-SNE
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PCA
scatter1 = axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='viridis')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('PCA')

# t-SNE
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='viridis')
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].set_title('t-SNE')

plt.colorbar(scatter2, ax=axes[1], label='Species')
plt.tight_layout()
plt.show()

---
## 7. モデルの保存と読み込み

In [ ]:
import pickle
import json

# モデルの学習
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print(f'元のモデル精度: {model.score(X_test, y_test):.4f}')

In [ ]:
# pickle で保存
with open('/tmp/model.pkl', 'wb') as f:
    pickle.dump(model, f)

# 読み込み
with open('/tmp/model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print(f'読み込んだモデル精度: {loaded_model.score(X_test, y_test):.4f}')

In [ ]:
# joblib を使う方法（推奨）
try:
    from joblib import dump, load
    
    # 保存
    dump(model, '/tmp/model.joblib')
    
    # 読み込み
    loaded_model = load('/tmp/model.joblib')
    print(f'joblib で読み込んだモデル精度: {loaded_model.score(X_test, y_test):.4f}')
except ImportError:
    print('joblib が利用できません')

### 練習問題 7

wine データセットを使って以下を実行してください。

1. StandardScaler、PCA（n_components=5）、RandomForestClassifier を含むパイプラインを作成
2. RandomForestのn_estimatorsを[50, 100, 200]でグリッドサーチ
3. 最良のモデルでテストデータを評価

In [ ]:
# 練習問題 7 の解答をここに書いてください


---
## まとめ

このチュートリアルで学んだ内容：

| トピック | 主なクラス・関数 |
|---------|---------------|
| アンサンブル学習 | `RandomForestClassifier`, `GradientBoostingClassifier`, `AdaBoostClassifier` |
| ハイパーパラメータ | `GridSearchCV`, `RandomizedSearchCV` |
| パイプライン | `Pipeline`, `make_pipeline`, `ColumnTransformer` |
| 特徴量選択 | `SelectKBest`, `RFE`, `VarianceThreshold` |
| クラスタリング | `KMeans`, `DBSCAN`, `silhouette_score` |
| 次元削減 | `PCA`, `TSNE` |
| モデル保存 | `pickle`, `joblib` |

---
## 練習問題の解答例

In [ ]:
# 練習問題 7 の解答例
print('--- 練習問題 7 ---')

from sklearn.datasets import load_wine
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

# データ
wine = load_wine()
X = wine.data
y = wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. パイプラインの作成
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),
    ('rf', RandomForestClassifier(random_state=42))
])

# 2. グリッドサーチ
param_grid = {
    'rf__n_estimators': [50, 100, 200]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(f'最良パラメータ: {grid_search.best_params_}')
print(f'最良CV精度: {grid_search.best_score_:.4f}')

# 3. テストデータの評価
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print(f'テスト精度: {test_score:.4f}')